In [2]:
import numpy as np
import pandas as pd

In [3]:
raw01 = pd.read_csv('lv2-2409.csv')
raw01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 13 columns):
PROD_ID    1541 non-null int64
P01        1541 non-null float64
P02        1541 non-null float64
P03        1541 non-null float64
P04        1541 non-null float64
P05        1541 non-null float64
P06        1541 non-null float64
P07        1541 non-null float64
P08        1541 non-null float64
P09        1541 non-null float64
P10        1541 non-null float64
P11        1541 non-null float64
DEFECT     1541 non-null int64
dtypes: float64(11), int64(2)
memory usage: 156.6 KB


In [4]:
list_fts = ['P01','P02','P03','P04','P05','P06','P07','P08','P09','P10','P11']

# 1

In [5]:
df210 = raw01.copy()

1-1

In [6]:
list_diff = []
for col in list_fts:
    seq00 = df210.loc[df210['DEFECT']==0, col].copy()
    seq01 = df210.loc[df210['DEFECT']==1, col].copy()
    Q01 = seq00.quantile(0.25)
    Q02 = seq00.quantile(0.5)
    Q03 = seq00.quantile(0.75)
    Q11 = seq01.quantile(0.25)
    Q12 = seq01.quantile(0.5)
    Q13 = seq01.quantile(0.75)
    diff = np.abs(Q01-Q11) + np.abs(Q02-Q12) + np.abs(Q03-Q13)
    list_diff.append(diff)

1-2

In [7]:
res21 = np.max(list_diff)
ans21 = int(round(res21, 0))
print(res21)
print('ans1 : ', ans21)

39792.254053404155
ans1 :  39792


# 2

In [8]:
df220 = raw01.copy()

2-1

In [9]:
for col in list_fts:
    max00 = df220[col].max()
    min00 = df220[col].min()
    df220[col] = (df220[col]-min00)/(max00-min00)

2-2

In [10]:
VAR2 = list(df220.loc[:, list_fts].std().sort_values(ascending=False)[:2].index)
VAR2

['P05', 'P11']

2-3

In [11]:
cor_val = df220.loc[:, VAR2].corr().iloc[0,1]
cor_abs = np.abs(cor_val)
print(cor_val)
print(cor_abs)

-0.12265890607171809
0.12265890607171809


2-4

In [12]:
res22 = cor_abs
ans22 = round(res22, 3)
print(res22)
print('ans2 : ', ans22)

0.12265890607171809
ans2 :  0.123


# 3

In [13]:
df231 = raw01.copy()

3-1

In [14]:
from sklearn.preprocessing import StandardScaler
scl00 = StandardScaler()
scl00.fit(df231.loc[:, list_fts])
df_tmp = pd.DataFrame(scl00.transform(df231.loc[:, list_fts]), columns=list_fts)
df_tmp.index = df231.index
df232 = pd.concat([df231.loc[:, ['PROD_ID','DEFECT']], df_tmp], axis=1)
df232

,PROD_ID,DEFECT,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11
0,1,1,-0.345026,-0.128324,-0.128324,0.155865,1.379296,-0.706791,-0.402089,-0.127122,-0.279032,0.375930,-2.025131
1,2,1,-1.231388,1.165291,1.165291,1.734877,-1.334561,-0.410250,-0.556946,-0.066122,-0.344642,0.455600,-2.077972
2,3,0,0.233036,-0.470819,-0.470819,-1.319912,-0.661792,1.098538,-0.164965,-0.081874,-0.328063,-0.499521,1.617660
3,4,1,-1.000163,0.687217,0.687217,0.919853,1.170809,-0.424050,-0.271429,-0.016537,-0.395260,-0.122011,0.409934
4,5,0,1.157935,-0.700994,-0.700994,0.510636,-0.651662,-0.048069,0.733527,-0.785159,0.776636,-0.981670,0.507530
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,1537,1,-1.539688,2.047366,2.047366,1.248581,0.728406,-0.478281,-1.425596,1.717534,-1.337761,1.786116,0.313194
1537,1538,1,-0.923088,0.555043,0.555043,2.680705,0.810400,-0.560779,-1.907103,2.726262,-1.563691,3.104692,-0.564929
1538,1539,1,-1.038701,0.758012,0.758012,0.050066,0.614619,-0.452795,-0.119798,-0.203511,-0.191218,-0.082580,-0.545577
1539,1540,0,-0.846013,0.434499,0.434499,0.687610,1.616511,2.540728,0.733527,-0.753410,0.704971,-0.760670,-0.227307


3-2

In [15]:
train23 = df232[df232['PROD_ID']%5!=0].copy()
test23 = df232[df232['PROD_ID']%5 == 0].copy()
print(train23.shape, test23.shape)

(1233, 13) (308, 13)


3-3, 3-4

In [16]:
from sklearn.neighbors import KNeighborsClassifier
list_acc = []
for k_ in [3, 5, 7]:
    knn00 = KNeighborsClassifier(metric = 'euclidean', n_neighbors=k_)
    knn00.fit(train23.loc[:, list_fts], train23['DEFECT'])
    arr_pred = knn00.predict(test23.loc[:, list_fts])
    arr_act = np.array(test23['DEFECT'])
    acc = np.sum(arr_pred==arr_act)/arr_pred.shape[0]
    list_acc.append(acc)
list_acc

[0.8993506493506493, 0.9090909090909091, 0.9025974025974026]

3-5

In [17]:
res23 = np.mean(list_acc)
ans23 = round(res23, 3)
print(res23)
print('ans3 : ', ans23)

0.9036796536796537
ans3 :  0.904


# summary

In [18]:
print('ans1 : ', ans21)
print('ans2 : ', ans22)
print('ans3 : ', ans23)

ans1 :  39792
ans2 :  0.123
ans3 :  0.904
